### read daily xls into pandas

In [1]:
from pandas import DataFrame, ExcelFile, concat
import os
from datetime import datetime
%matplotlib inline

In [2]:
path = ['C:/Users/Gérard/Dropbox/KGAST Daten/KGASTxls']
for i in path:
    try:
        dateiliste = os.listdir(i)
        pfad = i
        print('Erfolg mit: ', pfad)
    except:
        pass

# create list of all available files
xlss = [pfad + '/' + i for i in dateiliste if i[:5] == 'KGAST']

# read all available downloaded files
xls_list = []
for xls in xlss:
    df = ExcelFile(xls)
    xls_list.append(df.parse()[['MITGLIEDER', 'KURZ', 'VALOR', 'ANLAGEGRUPPE',
                      'AUSGABEPREIS', 'RUECKNAHMEPREIS', 'AUFGEL_ERTRAG',
                      'ABSCHLUSSTAG', 'VALUTA', 'AENDERUNGSDAT']])

Erfolg mit:  C:/Users/Gérard/Dropbox/KGAST Daten/KGASTxls


In [169]:
# create DataFrame, drop duplicates and change columns to lowercase
kurse = concat(xls_list).drop_duplicates()
kurse.columns = [i.lower() for i in kurse.columns]

def convert_to_float(zahl):
    ''' convert string to float else return None '''
    if isinstance(zahl, str):
        _zahl = zahl.replace("'","")
    else:
        _zahl = zahl
    try:
        return(float(_zahl))
    except ValueError:
        return None

def convert_to_datetime(datum):
    ''' convert string date to datetime '''
    try:
        return datetime.strptime(datum, '%Y-%m-%d')
        #return datetime(int(datum[:4]), int(datum[5:7]), int(datum[8:10]), 0, 0, 0, 0)
    except TypeError:
        return None #datetime(1900, 1, 1) # None

# convert to float
kurse.ausgabepreis = kurse.ausgabepreis.map(convert_to_float)
kurse.ruecknahmepreis = kurse.ruecknahmepreis.map(convert_to_float)
kurse.aufgel_ertrag = kurse.aufgel_ertrag.map(convert_to_float)

# convert to datetime format
kurse['abschlusstag'] = kurse.abschlusstag.map(convert_to_datetime)
kurse['valuta'] = kurse.valuta.map(convert_to_datetime)
kurse['aenderungsdat'] = kurse.aenderungsdat.map(convert_to_datetime)
# clean valor and convert to int
kurse.valor = kurse.valor.map(lambda x: int(str(x).replace(' ','')))

In [173]:
# delete Anlagestiftungen with few occurrences
few = 10
kurse_valor = DataFrame(kurse.groupby(by='valor').count())
valoren_del = kurse_valor[kurse_valor.mitglieder < few].index.tolist()
kurse3 = kurse[kurse.valor.map(lambda x: x not in valoren_del)]

In [174]:
kurse3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17608 entries, 0 to 433
Data columns (total 10 columns):
mitglieder         17608 non-null object
kurz               17608 non-null object
valor              17608 non-null int64
anlagegruppe       17608 non-null object
ausgabepreis       17471 non-null float64
ruecknahmepreis    17590 non-null float64
aufgel_ertrag      15485 non-null float64
abschlusstag       17606 non-null object
valuta             15923 non-null object
aenderungsdat      17606 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(1), object(5)
memory usage: 1.5+ MB


In [172]:
kurse.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17796 entries, 0 to 433
Data columns (total 10 columns):
mitglieder         17796 non-null object
kurz               17796 non-null object
valor              17796 non-null int64
anlagegruppe       17796 non-null object
ausgabepreis       17649 non-null float64
ruecknahmepreis    17766 non-null float64
aufgel_ertrag      15661 non-null float64
abschlusstag       17787 non-null object
valuta             16096 non-null object
aenderungsdat      17784 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(1), object(5)
memory usage: 1.5+ MB


In [ ]:
valoren_liste = k.valor.drop_duplicates()
valor = valoren_liste[12]